In [1]:
import matplotlib.pyplot
import pandas
import numpy
import os

In [2]:
def open_drone_logs():
    drones = {}
    for file in os.listdir('../out/drones'):
        ID = file.replace('.csv', '')
        path = os.path.join('../out/drones', file)
        df = pandas.read_csv(path)
        drones[ID] = df
    return drones

In [3]:
def open_targets_log():
    return pandas.read_csv('../out/targets.csv')

In [4]:
def get_distance(A: pandas.DataFrame, B: pandas.DataFrame):
    return numpy.linalg.norm((numpy.array(A[['PosX', 'PosY', 'PosZ']])) - (numpy.array(B[['PosX', 'PosY', 'PosZ']])), axis=1)

def get_distances(drones: dict):
    return {ID_A: {ID_B:get_distance(drones[ID_A], drones[ID_B]) for ID_B in drones if ID_B != ID_A} for ID_A in drones}

def get_min_distances(distances: dict):
    return {ID:numpy.min(numpy.array(list(distances[ID].values())), axis=0) for ID in distances}

def get_max_distances(distances: dict):
    return {ID:numpy.max(numpy.array(list(distances[ID].values())), axis=0) for ID in distances}

In [5]:
drones = open_drone_logs()
targets = open_targets_log()

In [6]:
distances = get_distances(drones)
min_distances = get_min_distances(distances)
max_distances = get_max_distances(distances)
for ID in drones:
    drones[ID]['MinDistanceFromDrones'] = min_distances[ID]
    drones[ID]['MaxDistanceFromDrones'] = max_distances[ID]

In [7]:
def plotter(func):
    def wrapper(*args, **kwargs):
        fig = matplotlib.pyplot.figure(figsize=(10, 10))
        func(fig, *args, **kwargs)
        matplotlib.pyplot.legend()
        matplotlib.pyplot.savefig(func.__qualname__)
        matplotlib.pyplot.close()
    return wrapper

In [8]:
def plot_drone_metrics(ID: str, df: pandas.DataFrame, x: str, y: str):
    matplotlib.pyplot.plot(df[x], df[y], label=ID)

def plot_drones_metrics(fig, drones: dict, x: str, y: str):
    fig = matplotlib.pyplot.figure(figsize=(20, 20))
    for ID in drones:
        plot_drone_metrics(ID, drones[ID], x, y)
    matplotlib.pyplot.xlabel(x)
    matplotlib.pyplot.ylabel(y)

@plotter
def plot_drones_distance_from_target_over_time(fig, drones: dict):
    plot_drones_metrics(fig, drones, 'Timestamp', 'DistanceFromTarget')

@plotter
def plot_drones_speed_over_time(fig, drones: dict):
    plot_drones_metrics(fig, drones, 'Timestamp', 'Speed')

@plotter
def plot_drones_min_distance_from_drones_over_time(fig, drones: dict):
    plot_drones_metrics(fig, drones, 'Timestamp', 'MinDistanceFromDrones')

@plotter
def plot_drones_max_distance_from_drones_over_time(fig, drones: dict):
    plot_drones_metrics(fig, drones, 'Timestamp', 'MaxDistanceFromDrones')

In [9]:
drones['sp0']

,Timestamp,PosX,PosY,PosZ,Target,DistanceFromTarget,Speed,TaskAllocatorState,TaskExecutorState,MinDistanceFromDrones,MaxDistanceFromDrones
0,1,-0.570282,3.04124,0.00000,0,6.13297,0.000000,1,0,0.855728,13.196430
1,2,-0.570282,3.04124,0.00000,0,6.13297,0.000000,1,0,0.855728,13.196430
2,3,-0.570282,3.04124,0.00000,0,6.13297,0.000000,1,0,0.855728,13.196430
3,4,-0.570282,3.04124,0.00000,0,6.13297,0.000000,1,0,0.855728,13.196430
4,5,-0.570282,3.04124,0.00000,0,6.13297,0.000000,1,0,0.855728,13.196430
...,...,...,...,...,...,...,...,...,...,...,...
1492,1493,4.496370,3.79216,2.81723,0,1.28336,0.149633,1,4,0.297126,15.391900
1493,1494,4.496720,3.79207,2.81726,0,1.28336,0.149633,1,4,0.296817,15.392074
1494,1495,4.497080,3.79198,2.81730,0,1.28336,0.149633,1,4,0.296501,15.392249
1495,1496,4.497430,3.79189,2.81733,0,1.28336,0.149633,1,4,0.296199,15.392424


In [10]:
plot_drones_distance_from_target_over_time(drones)
plot_drones_speed_over_time(drones)
plot_drones_min_distance_from_drones_over_time(drones)
plot_drones_max_distance_from_drones_over_time(drones)

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>